**Table of contents**<a id='toc0_'></a>    
- [Import Raw CSV Files, Concatenate into one Dataframe:]    
    - [Within same loop that reads in csvs, append `city` and `state` columns based on file name:]
- [Clean `city` and `state` Columns:]    
    - [Remove `.csv` from end of `state` column:]    
    - [Replace dashes with spaces in `city` and `state` columns:]    
  - [Check Los Angeles `id` Col for Sci Notation:]
  - [Check that all city names came in:]
  - [Check that all state names came in:]
- [Remove Line Breaks and other special characters]
    - [Success! There are no line breaks in the text above.]
- [Explore Duplicates:]
  - [Duplicates on `ID`]
    - [Drop the above duplicate Columns:]
  - [Check Again after Dropping:]
  - [Duplicates on All Rows except `id`?]
  - [Duplicates on `description`]
    - [Duplicate Descriptions - Findings:]
- [Duplicates on listing_url]
  - [Check for overall Duplicates:]
  - [There are 0 Duplicate rows to drop:]
- [Remove irrelevant columns and those that do not add predictive value]
- [Reset Index before exporting to Git:]
- [Export data with dropped columns to Git Repo:]
  - [Import CSV stored in GitHub to Google Colab]

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

<a href="https://colab.research.google.com/github/clairesarraille/airbnb_price_prediction/blob/main/data_cleaning_ver04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
# Core Packages:
import pandas as pd
import pickle
import numpy as np

"""
# Google Colab:
### Print Dataframe like Spreadsheet!
from google.colab import data_table
data_table.enable_dataframe_formatter()
#"""

"""
# Mount Drive:
from google.colab import drive
drive.mount('/content/drive')
#"""



"\n# Mount Drive:\nfrom google.colab import drive\ndrive.mount('/content/drive')\n#"

# <a id='toc1_'></a>[Import Raw CSV Files, Concatenate into one Dataframe:](#toc0_)
- Raw CSV filenames follow the convention `CITY_STATE.csv`
- Use `glob` to read all `.csv` filenames from folder `listing_raw_data`
- Append `city` and `state` columns to each dataframe and append into one called `df`

In [37]:
import pandas as pd
import glob
import os
os.chdir("/Users/claire/git/flatiron/capstone/Data/listing_raw_data")
#os.chdir("/Users/claire/git/flatiron/capstone/Data/data_no_la")

filenames = [i for i in glob.glob('*.csv')]

### <a id='toc1_1_1_'></a>[Within same loop that reads in csvs, append `city` and `state` columns based on file name:](#toc0_)

In [38]:
# Read in each csv by iterating through filenames,
# appending city and state columns based on csv name (delmited with underscore)
df = pd.concat((pd.read_csv(name).assign(city = name.split('_')[0], state = name.split('_')[1]) for name in filenames))

### <a id='toc1_1_2_'></a>[Preview Columns:](#toc0_)

In [39]:
pd.set_option('display.max_colwidth', 70)
df.head(3)

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,city,state
0,3314819,https://www.airbnb.com/rooms/3314819,20230913045235,2023-09-13,city scrape,Home in Asheville · ★4.75 · 1 bedroom · 1 bed · 1 private bath,Don’t pay ridiculously high cleaning fees elsewhere! We keep fees ...,South slope of Brown Mountain overlooking south Asheville toward M...,https://a0.muscache.com/pictures/42052824/3b0cc939_original.jpg,16296123,...,4.88,NaN,f,3,0,3,0,10.99,Asheville,North-Carolina.csv
1,108061,https://www.airbnb.com/rooms/108061,20230913045235,2023-09-13,city scrape,Rental unit in Asheville · ★4.51 · 1 bedroom · 1 bed · 1 bath,Walk to town in ten minutes! Monthly rental includes all utilities...,"I love my neighborhood! Its friendly, easy-going, quiet and the cl...",https://a0.muscache.com/pictures/miso/Hosting-108061/original/9f3e...,320564,...,4.49,NaN,f,2,2,0,0,0.63,Asheville,North-Carolina.csv
2,155305,https://www.airbnb.com/rooms/155305,20230913045235,2023-09-13,city scrape,Guesthouse in Asheville · ★4.59 · 1 bedroom · 1 bed · 1 bath,<b>The space</b><br />Pet friendly private cottage located behind ...,"We are within easy walk of pubs, breweries, music, thrift shops, g...",https://a0.muscache.com/pictures/8880711/cf38d21c_original.jpg,746673,...,4.54,NaN,f,7,1,2,4,2.71,Asheville,North-Carolina.csv


### <a id='toc1_1_3_'></a>[Comments on Column Preview:](#toc0_)
- Cols appear to have imported correctly
- City and State Columns have been imported -- need to remove the ".csv" from state and replace dashes with spaces

# <a id='toc2_'></a>[Clean `city` and `state` Columns:](#toc0_)

### <a id='toc2_1_1_'></a>[Remove `.csv` from end of `state` column:](#toc0_)

In [40]:
df['state'] = df['state'].replace('.csv','', regex=True)

### <a id='toc2_1_2_'></a>[Replace dashes with spaces in `city` and `state` columns:](#toc0_)


In [41]:
df[['city', 'state']].loc[df['city'] == 'Los-Angeles'].head(3)

,city,state
0,Los-Angeles,California
1,Los-Angeles,California
2,Los-Angeles,California


In [42]:
df[['city', 'state']].loc[df['city'] == 'Asheville'].head(3)

,city,state
0,Asheville,North-Carolina
1,Asheville,North-Carolina
2,Asheville,North-Carolina


In [43]:
df['state'] = df['state'].replace('-',' ', regex=True)
df['city'] = df['city'].replace('-',' ', regex=True)

In [44]:
df[['city', 'state']].loc[df['city'] == 'Los Angeles'].head(3)

,city,state
0,Los Angeles,California
1,Los Angeles,California
2,Los Angeles,California


In [45]:
df[['city', 'state']].loc[df['city'] == 'Asheville'].head(3)

,city,state
0,Asheville,North Carolina
1,Asheville,North Carolina
2,Asheville,North Carolina


## <a id='toc2_2_'></a>[Check Los Angeles `id` Col for Sci Notation:](#toc0_)

In [46]:
# Make sure id column didn't revert to scientific notation for Los Angeles (this was a concern b/c file had macros)
df[['id']].loc[df['city'] == 'Los Angeles'].sort_values(by = 'id',ascending=False).head(10)

,id
33827,972398639579333846
44177,971820943924237257
24095,971811914654853933
13467,971788297972650723
28010,971677504036551235
12316,971642544313353359
6878,971629449015511902
36359,971624556820195385
5510,971580325208006627
39010,971497265580348153


## <a id='toc2_3_'></a>[Check that all city names came in:](#toc0_)

In [47]:
print(f"Unique number of citites in this dataset: {len(df['city'].unique())}")
print()
for city in df['city'].unique():
    print(city)

Unique number of citites in this dataset: 32

Asheville
Hawaii
Dallas
San Mateo County
Portland
Oakland
Chicago
New Orleans
Bozeman
Los Angeles
Rochester
Newark
New York City
Boston
Washington D.C.
Fort Worth
Rhode Island
Santa Clara County
Cambridge
Pacific Grove
Santa Cruz County
Salem
Columbus
Nashville
Austin
Denver
Twin Cities
San Diego
Broward County
Seattle
Clark County
Jersey City


## <a id='toc2_4_'></a>[Check that all state names came in:](#toc0_)

In [48]:
print(f"Unique number of states in this dataset: {len(df['city'].unique())}")
print()
for state in df['state'].unique():
    print(state)

Unique number of states in this dataset: 32

North Carolina
Hawaii
Texas
California
Oregon
Illinois
Louisiana
Montana
New York
New Jersey
Massachusetts
District of Columbia
Rhode Island
Ohio
Tennessee
Colorado
Minnesota
Florida
Washington
Nevada


# <a id='toc3_'></a>[Remove Line Breaks and other special characters](#toc0_)
- There were many different special characters that were causing column off-setting in exported csv
- When I inspected the csv I exported to make available in Google Colab, I found that after text columns such as "host_about"...
- A bunch of columns would be blank, which off-set the values of future columns
- I removed ALL special characters from all fields except whitespace and `-_,.` (dash, underscore, comma, period)

In [49]:
# Python "object" dtype is tricky - make sure to include it for .replace
# I can avoid converting all text columns to dtype "string" first ny using "include='object'"
for i in df.select_dtypes(include='object').columns:
    df[i]=df[i].str.replace('[ ](?=[ ])|[^-_,.A-Za-z0-9 ]+','', regex=True)

In [50]:
# Check a sample row that I found broke the exported csv because it contained newline characters
# (including hidden chars not visible in pandas)
pd.set_option('display.max_colwidth', 3000)
df['host_about'].loc[df['host_about'].str.contains('We live in Asheville in the beautiful mountains of western North Carolina', na=False)].head(20)

0       Hello My husband and I are looking forward to hosting guests in our home as well as being a guest of others during our travels. We became AirBnB hosts in 2014 and have since hosted more than 2,000 bookings. In 2018, we became AirBnB superhostsWe live in Asheville in the beautiful mountains of western North Carolina.Allison works as a fourth grade teacher at Mills River Elementary School. Peter works as the Trails Coordinator at a regional land trust, Conserving Carolina. Both of us work to keep this region we love as wonderful as it is today, and make it even better for future generations.We love to take road trips throughout the year. We have appreciated the kindness of many over the years who have let us stay in their home or camp in their yard, or have rented a room to us for a reasonable rate. We are guests who are focused on our travels and we most often are seeking simply a room to hang our hat for the night before our travels resume the following morning. For that reason

### <a id='toc3_1_1_'></a>[Success! There are no line breaks in the text above.](#toc0_)

# <a id='toc4_'></a>[Explore Duplicates:](#toc0_)

## <a id='toc4_1_'></a>[Duplicates on `ID`](#toc0_)

In [51]:
# There is one listing that's duplicated on 'id' and it also has the same URL, so we can safely remove this
pd.set_option('display.max_colwidth', 100)
df[df.duplicated(['id'], keep=False)]


,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,city,state
47,1398981,httpswww.airbnb.comrooms1398981,20230918050432,2023-09-18,city scrape,Bungalow in Palo Alto 4.81 1 bedroom 2 beds 1 bath,"2 bedroom, 1 bathroom, cozy kitchen and access to laundry room. br Youll love my place because o...","Please note my home is located on the Westside of Palo Alto, I am on the border of Menlo park an...",httpsa0.muscache.compicturesf2d6bbfd-dac5-4379-a86a-10ecbb28c982.jpg,52835,...,4.73,NaN,f,1,1,0,0,0.3,San Mateo County,California
79,1398981,httpswww.airbnb.comrooms1398981,20230918144140,2023-09-18,city scrape,Bungalow in Palo Alto 4.81 1 bedroom 2 beds 1 bath,"2 bedroom, 1 bathroom, cozy kitchen and access to laundry room. br Youll love my place because o...","Please note my home is located on the Westside of Palo Alto, I am on the border of Menlo park an...",httpsa0.muscache.compicturesf2d6bbfd-dac5-4379-a86a-10ecbb28c982.jpg,52835,...,4.73,NaN,f,1,1,0,0,0.3,Santa Clara County,California


### <a id='toc4_1_1_'></a>[Drop the above duplicate Columns:](#toc0_)

In [52]:
df = df.drop_duplicates(subset=['id'])

## <a id='toc4_2_'></a>[Check Again after Dropping:](#toc0_)

In [53]:
df[df.duplicated(['id'], keep=False)]

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,city,state


## <a id='toc4_3_'></a>[Duplicates on All Rows except `id`?](#toc0_)
- Check for rows where all values are duplicated EXCEPT `id`:


In [54]:
dupe_desc = pd.DataFrame(df.drop(['id'],axis=1).values)

In [55]:
print(type(dupe_desc))

<class 'pandas.core.frame.DataFrame'>


In [56]:
print(df[df.duplicated()])
dupe_desc[dupe_desc.duplicated()]

Empty DataFrame
Columns: [id, listing_url, scrape_id, last_scraped, source, name, description, neighborhood_overview, picture_url, host_id, host_url, host_name, host_since, host_location, host_about, host_response_time, host_response_rate, host_acceptance_rate, host_is_superhost, host_thumbnail_url, host_picture_url, host_neighbourhood, host_listings_count, host_total_listings_count, host_verifications, host_has_profile_pic, host_identity_verified, neighbourhood, neighbourhood_cleansed, neighbourhood_group_cleansed, latitude, longitude, property_type, room_type, accommodates, bathrooms, bathrooms_text, bedrooms, beds, amenities, price, minimum_nights, maximum_nights, minimum_minimum_nights, maximum_minimum_nights, minimum_maximum_nights, maximum_maximum_nights, minimum_nights_avg_ntm, maximum_nights_avg_ntm, calendar_updated, has_availability, availability_30, availability_60, availability_90, availability_365, calendar_last_scraped, number_of_reviews, number_of_reviews_ltm, number_of_

,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,75


## <a id='toc4_4_'></a>[Duplicates on `description`](#toc0_)

In [57]:
pd.set_option('display.max_colwidth', 3000)

In [58]:
inspect_dupes = df[['id', 'description', 'listing_url']][df.duplicated(['description'])]

In [59]:
inspect_dupes.sort_values(by='description').head()

,id,description,listing_url
33479,867036314829984141,,httpswww.airbnb.comrooms867036314829984141
41108,927935810987192242,,httpswww.airbnb.comrooms927935810987192242
8864,727747618102486531,,httpswww.airbnb.comrooms727747618102486531
2284,961709908898772907,,httpswww.airbnb.comrooms961709908898772907
41091,937200163326307413,,httpswww.airbnb.comrooms937200163326307413


### <a id='toc4_4_1_'></a>[Duplicate Descriptions - Findings:](#toc0_)
- These tend to be rooms in hotel-like complexes -- which would be good to remove but may be a more intuitive way to do so, like room type.

# <a id='toc5_'></a>[Duplicates on listing_url](#toc0_)
- There are 0 duplicated on `listing_url`

In [60]:
len(df)

274958

In [61]:
df[['listing_url']].nunique()

listing_url    274958
dtype: int64

In [62]:
inspect_dupes = df[['id', 'description', 'listing_url']][df.duplicated(['listing_url'])]

## <a id='toc5_1_'></a>[Check for overall Duplicates:](#toc0_)
- There are 0 entire row duplicates:

In [63]:
print(df[df.duplicated()])

print(len(df))
print(len(df.duplicated()))

Empty DataFrame
Columns: [id, listing_url, scrape_id, last_scraped, source, name, description, neighborhood_overview, picture_url, host_id, host_url, host_name, host_since, host_location, host_about, host_response_time, host_response_rate, host_acceptance_rate, host_is_superhost, host_thumbnail_url, host_picture_url, host_neighbourhood, host_listings_count, host_total_listings_count, host_verifications, host_has_profile_pic, host_identity_verified, neighbourhood, neighbourhood_cleansed, neighbourhood_group_cleansed, latitude, longitude, property_type, room_type, accommodates, bathrooms, bathrooms_text, bedrooms, beds, amenities, price, minimum_nights, maximum_nights, minimum_minimum_nights, maximum_minimum_nights, minimum_maximum_nights, maximum_maximum_nights, minimum_nights_avg_ntm, maximum_nights_avg_ntm, calendar_updated, has_availability, availability_30, availability_60, availability_90, availability_365, calendar_last_scraped, number_of_reviews, number_of_reviews_ltm, number_of_

## <a id='toc5_2_'></a>[There are 0 Duplicate rows to drop:](#toc0_)

In [64]:
len(df)-len(df.drop_duplicates())

0

# <a id='toc6_'></a>[Remove irrelevant columns and those that do not add predictive value](#toc0_)
- Remove columns that don't add predictive value, or are irrelevant because brand-new listings created by first-time listers will be NULL when entered into model:
- See "data_understanding.ipynb" for justifications to drop cols
- Also remove columns that a first-time lister who has ONE listing wouldn't have, such as reviews and response rate, etc

In [65]:
col_list = ['id', 'name', 'description', 'neighborhood_overview', 'host_location', 'host_about', 'host_neighbourhood', 'host_verifications', 'host_has_profile_pic', 'host_identity_verified', 'neighbourhood', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude', 'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price', 'minimum_nights', 'maximum_nights', 'has_availability', 'availability_30', 'availability_60', 'availability_90', 'availability_365', 'license', 'instant_bookable', 'city', 'state']
df = df[col_list]

# <a id='toc7_'></a>[Reset Index before exporting to Git:](#toc0_)
- We don't necessarily need to do this, but it's good housekeeping.

In [66]:
df = df.reset_index(drop=True)

In [67]:
pd.set_option('display.max_colwidth', 70)
df.head(3)

,id,name,description,neighborhood_overview,host_location,host_about,host_neighbourhood,host_verifications,host_has_profile_pic,host_identity_verified,...,maximum_nights,has_availability,availability_30,availability_60,availability_90,availability_365,license,instant_bookable,city,state
0,3314819,Home in Asheville 4.75 1 bedroom 1 bed 1 private bath,Dont pay ridiculously high cleaning fees elsewhere We keep fees lo...,South slope of Brown Mountain overlooking south Asheville toward M...,"Asheville, NC",Hello My husband and I are looking forward to hosting guests in ou...,NaN,"email, phone",t,t,...,7,t,17,39,66,66,NaN,f,Asheville,North Carolina
1,108061,Rental unit in Asheville 4.51 1 bedroom 1 bed 1 bath,"Walk to town in ten minutes Monthly rental includes all utilities,...","I love my neighborhood Its friendly, easy-going, quiet and the clo...","Asheville, NC",I am a long time resident of Asheville and am glad help you find g...,NaN,"email, phone",t,t,...,365,t,0,2,32,203,NaN,f,Asheville,North Carolina
2,155305,Guesthouse in Asheville 4.59 1 bedroom 1 bed 1 bath,bThe spacebbr Pet friendly private cottage located behind the main...,"We are within easy walk of pubs, breweries, music, thrift shops, g...","Asheville, NC","We operate a travelers hostel located in Historic West Asheville, ...",NaN,"email, phone",t,t,...,365,t,22,39,65,268,NaN,f,Asheville,North Carolina


# <a id='toc8_'></a>[Export data with dropped columns to Git Repo:](#toc0_)
- Export Cleaned Data to CSV:
- For this step, you'll need to have Git Large File Service installed: https://git-lfs.com/

In [68]:
airbnb_path = '/Users/claire/git/flatiron/capstone/airbnb_price_prediction/airbnb_data.csv'
df.to_csv(airbnb_path, encoding='utf-8-sig', index=False)

## <a id='toc8_1_'></a>[Import CSV stored in GitHub to Google Colab](#toc0_)
- The remaining code must be executed in Google Colab and may require a professional subscription to run.
- Code for importing csvs from GitHub to Google Colab:

In [69]:
# To get this URL, so to the csv file on GitHub, choose "Blame", and then click on the "raw" link
# url = 'https://media.githubusercontent.com/media/clairesarraille/airbnb_price_prediction/main/airbnb_data.csv?token=ARCBHGLTVY3OAQ27BXQQBRDFJPI4K'
# df = pd.read_csv(url)